In [1]:
import numpy as np
import matplotlib.pyplot
from randomNN import randomNN
from pyBO import pyBO

pyBO version: 1.0.1. updated on 2024-03-02


In [ ]:
benchmark_hyper_params = {}
ndim = 2
benchmark_hyper_params[ndim] = {}
for budget in [20,30,40]:
    benchmark_hyper_params[ndim][budget] = []
    for n_init in [int(0.2*budget),int(0.3*budget),int(0.4*budget)]:
        for ibetadecay in [None, n_init +int(0.3*(budget-n_init)),  n_init +int(0.6*(budget-n_init))]:
            for imovingWindow in [None, n_init +int(0.3*(budget-n_init)),  n_init +int(0.6*(budget-n_init))]:
                for isafebeta in [None, n_init +int(0.3*(budget-n_init)),  n_init +int(0.6*(budget-n_init))]:
                    benchmark_hyper_params[ndim][budget].append(budget,n_init,ibetadecay,imovingWindow,isafebeta)
ndim = 4
benchmark_hyper_params[ndim] = {}
for budget in [40,60,80]:
    benchmark_hyper_params[ndim][budget] = []
    for n_init in [int(0.2*budget),int(0.3*budget),int(0.4*budget)]:
        for ibetadecay in [None, n_init +int(0.3*(budget-n_init)),  n_init +int(0.6*(budget-n_init))]:
            for imovingWindow in [None, n_init +int(0.3*(budget-n_init)),  n_init +int(0.6*(budget-n_init))]:
                for isafebeta in [None, n_init +int(0.3*(budget-n_init)),  n_init +int(0.6*(budget-n_init))]:
                    benchmark_hyper_params[ndim][budget].append(budget,n_init,ibetadecay,imovingWindow,isafebeta)
ndim = 6
benchmark_hyper_params[ndim] = {}
for budget in [60,100,140]:
    benchmark_hyper_params[ndim][budget] = []
    for n_init in [int(0.2*budget),int(0.3*budget),int(0.4*budget)]:
        for ibetadecay in [None, n_init +int(0.3*(budget-n_init)),  n_init +int(0.6*(budget-n_init))]:
            for imovingWindow in [None, n_init +int(0.3*(budget-n_init)),  n_init +int(0.6*(budget-n_init))]:
                for isafebeta in [None, n_init +int(0.3*(budget-n_init)),  n_init +int(0.6*(budget-n_init))]:
                    benchmark_hyper_params[ndim][budget].append(budget,n_init,ibetadecay,imovingWindow,isafebeta)

# randomNN var is 1 if input var is 1

In [2]:
for ndim in [2,4,6,8,16]:
    vm = randomNN(ndim,1)
    x = np.random.randn(1024*8,ndim)
    print(ndim,np.std(vm(x)))

2 1.005450336671933
4 1.0635085671919413
6 0.9820406719692341
8 0.9951638265008312
16 1.0301106204722568


# benchmark

In [3]:
def benchmark_1(fun, ndim, budget, n_init, ibetadecay=None, imovingWindow=None, isafebeta=None, beta0=9):
    bounds = np.array([(-3,3) for i in range(ndim)])
    bounds_diff = bounds[:,1] - bounds[:,0]
    bo, X_pending, Y_pending_future = pyBO.runBO(
                                        fun,  
                                        bounds = bounds,
                                        n_init = n_init,
                                        budget = n_init+1,
                                        batch_size=1,
                                        )
    while(len(bo.y)<budget):
        beta = beta0
        if ibetadecay:
            if len(bo.y) > ibetadecay:
                beta = max(1, 1  +(beta0-1)*(1-(len(bo.y)-ibetadecay)/(budget-ibetadecay) ) )
        acquisition_func_args={'beta':beta}
        if isafebeta:
            if len(bo.y) > isafebeta:
                acquisition_func_args['safe_beta'] = 9
        if imovingWindow:
            if len(bo.y) > imovingWindow:
                x_best,y_best = bo.best_sofar()
                bounds = np.array(list(zip(x_best-0.1*bounds_diff, x_best+0.1*bounds_diff)))
        X_pending, Y_pending_future= bo.loop( 
                            n_loop=1,  # number of additional optimization interation
                            func_obj = fun,
                            bounds = bounds,
                            acquisition_func_args = acquisition_func_args,
                            X_pending = X_pending, 
                            Y_pending_future = Y_pending_future,
                            batch_size = 1,
                            )
    x_best,y_best = bo.best_sofar()
    return y_best

In [ ]:
ndim = 2
benchmark_hyper_params[ndim] = {}
for budget in [20,30,40]:
    benchmark_hyper_params[ndim][budget] = []
    for n_init in [int(0.2*budget),int(0.3*budget),int(0.4*budget)]:
        for ibetadecay in [None, n_init +int(0.3*(budget-n_init)),  n_init +int(0.6*(budget-n_init))]:
            for imovingWindow in [None, n_init +int(0.3*(budget-n_init)),  n_init +int(0.6*(budget-n_init))]:
                for isafebeta in [None, n_init +int(0.3*(budget-n_init)),  n_init +int(0.6*(budget-n_init))]:
                    benchmark_hyper_params[ndim][budget].append(budget,n_init,ibetadecay,imovingWindow,isafebeta)